In [7]:
import os
import numpy as np
import pandas as pd

In [8]:
ORI_DATA = 'src_data/mpox_2quick_weekly_pgraph.csv'
OUT_DATA = 'src_data/cbg2cbg.csv'

In [19]:
fout = open(OUT_DATA, 'w')
with open(ORI_DATA, 'r') as f:
    for line in f.readlines():
        if 'CA' not in line:
            fout.write(line)
fout.close()

In [32]:
OUT_DIR = 'src_data/cbg_data/'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

In [36]:
with open(OUT_DIR + 'cbg_name', 'w') as fout:
    isheader = True
    cbgNames = set()
    with open(OUT_DATA, 'r') as f:
        for line in f.readlines():
            if isheader:
                isheader = False
                continue
            cbgNames.add(line.strip().split(',')[0])
            cbgNames.add(line.strip().split(',')[1])
    cbgNames = list(cbgNames)
    for cbg in cbgNames:
        fout.write(cbg + '\n')

In [42]:
OUT_DIR2 = 'src_data/cbg_data/cbg2cbg/'
if not os.path.exists(OUT_DIR2):
    os.makedirs(OUT_DIR2)

In [13]:
import random

In [26]:
def get_ramdom_dest(src_df):
    randval = random.random()
    cumprob = src_df['probability'].cumsum()
    sumprob = src_df['probability'].sum()
    if sumprob == 0:
        return 
    randval *= sumprob
    checkprob = 0
    for des, prob in zip(src_df['poi_cbg_destination'], cumprob):
        if checkprob == prob:
            continue
        if randval < prob:
            return des
    return src_df.loc[-1,'poi_cbg_destination']

In [6]:
def recalculate(group):
    group = group.copy()
    group.drop(columns=['poi_cbg_source'], inplace=True)

    sumprob = group['probability'].sum()
    if abs(sumprob) < 1e-18:  # Handle very small sums close to zero
        return group
    group['probability'] = group['probability'].div(sumprob)

    return group

In [10]:
chunked_data = pd.read_csv(OUT_DATA, chunksize=100000)
chunk_list = []
for chunk in chunked_data:
    chunk_list.append(chunk)
df = pd.concat(chunk_list)

In [11]:
df

,poi_cbg_source,poi_cbg_destination,number_of_visitors,probability
0,10010201001,10010201001,33,0.038239
1,10010201001,10010202001,15,0.017381
2,10010201001,10010202002,36,0.041715
3,10010201001,10010203002,113,0.130939
4,10010201001,10010204003,6,0.006952
...,...,...,...,...
17423096,560459513003,560279572003,6,0.008876
17423097,560459513003,560459511002,3,0.004438
17423098,560459513003,560459513001,69,0.102071
17423099,560459513003,560459513002,229,0.338757


In [25]:
get_ramdom_dest(df.groupby('poi_cbg_source').get_group(10010201001))

0.08194242474010716
0.9826187717265332
0.08051816475041798
10010201001 0.0382387022016222
10010202001 0.0382387022016222
10010202002 0.0799536500579373
10010203002 0.21089223638470428


10010203002

In [17]:
remove_idx = df[df['poi_cbg_destination'] == 10010202001].index
df.loc[remove_idx, 'probability'] = 0
df

,poi_cbg_source,poi_cbg_destination,number_of_visitors,probability
0,10010201001,10010201001,33,0.038239
1,10010201001,10010202001,15,0.000000
2,10010201001,10010202002,36,0.041715
3,10010201001,10010203002,113,0.130939
4,10010201001,10010204003,6,0.006952
...,...,...,...,...
17423096,560459513003,560279572003,6,0.008876
17423097,560459513003,560459511002,3,0.004438
17423098,560459513003,560459513001,69,0.102071
17423099,560459513003,560459513002,229,0.338757
